In [21]:
# Install & Imports
!pip install pymoo==0.5.0

import pandas as pd
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.factory import get_termination
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting

In [22]:
# Load Dataset
file_path = "Dummy_Lawyers.csv"
df = pd.read_csv(file_path)

In [23]:
# Define Optimization Problem
class LawyerSelectionProblem(Problem):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        super().__init__(n_var=len(df), n_obj=4, n_constr=0, xl=0.0, xu=1.0)

    def _evaluate(self, X, out, *args, **kwargs):
        objs = []
        for weights in X:
            weights = np.clip(weights, 0, 1)
            weights /= np.sum(weights) + 1e-6  # normalize weights

            profile = (self.df[[
                "Years of active experience",
                "No of favoured settlements",
                "Client satisfaction (out of 10)",
                "Price"
            ]].T @ weights).values

            objs.append([
                -profile[0],  # maximize experience
                -profile[1],  # maximize settlements
                -profile[2],  # maximize satisfaction
                profile[3]    # minimize price
            ])
        out["F"] = np.array(objs)

In [24]:
# Run NSGA-2
def recommend_nsga2(domain, budget, seed=1, generations=100, pop_size=50):
    # Filter dataset
    filtered_df = df[(df["Domain"] == domain) & (df["Price"] <= budget)].reset_index(drop=True)
    if filtered_df.empty:
        print("No matching lawyers found for given filters.")
        return pd.DataFrame()

    # Define NSGA-II problem
    problem = LawyerSelectionProblem(filtered_df)
    algorithm = NSGA2(
        pop_size=pop_size,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True
    )
    termination = get_termination("n_gen", generations)

    # Run optimization
    res = minimize(problem, algorithm, termination, seed=seed, verbose=False)

    # Get Pareto-optimal indices
    pareto_idx = NonDominatedSorting().do(res.F, only_non_dominated_front=True)

    # Collect lawyers corresponding to Pareto front
    pareto_lawyers = []
    for i in pareto_idx:
        weights = np.clip(res.X[i], 0, 1)
        weights /= np.sum(weights) + 1e-6
        idx = np.argmax(weights)
        lawyer = filtered_df.iloc[idx].to_dict()
        lawyer["Objective Vector"] = res.F[i]
        lawyer["Price Objective"] = res.F[i][3]
        pareto_lawyers.append(lawyer)

    # Create DataFrame
    result_df = pd.DataFrame(pareto_lawyers).drop_duplicates(subset=["Name"])
    result_df = result_df.sort_values(by="Price Objective").reset_index(drop=True)

    # Show relevant columns
    return result_df[[
        "Name", "Domain", "Price", "Years of active experience",
        "No of favoured settlements", "Client satisfaction (out of 10)", "Price Objective"
    ]]

In [25]:
# Example 1
print("=== Civil Law, Budget 20000 ===")
display(recommend_nsga2("Civil Law", 20000))

# Example 2
print("=== Tax Law, Budget 30000 ===")
display(recommend_nsga2("Tax Law", 30000))

# Example 3
print("=== Corporate Law, Budget 50000 ===")
display(recommend_nsga2("Corporate Law", 50000))

=== Civil Law, Budget 20000 ===


,Name,Domain,Price,Years of active experience,No of favoured settlements,Client satisfaction (out of 10),Price Objective
0,Autumn Scott,Civil Law,7017.60,14,39,4.2,11092.332908
1,Juan Kaiser,Civil Law,19032.32,36,26,6.3,12408.713588
2,Patricia Cameron,Civil Law,8718.26,8,40,5.4,12741.773710
3,Kimberly Garza,Civil Law,16347.37,16,153,4.2,12745.236453
4,James Arnold,Civil Law,17657.24,2,184,6.3,13682.400364
5,Daniel Figueroa,Civil Law,14729.53,35,44,4.4,14577.830895
6,Nancy Richardson,Civil Law,11867.94,25,52,4.6,15013.970284
7,Timothy Landry,Civil Law,19199.69,19,0,9.4,15357.435229


=== Tax Law, Budget 30000 ===


,Name,Domain,Price,Years of active experience,No of favoured settlements,Client satisfaction (out of 10),Price Objective
0,Lindsay Davis,Tax Law,13352.59,18,100,4.1,17969.237112
1,Angela Martin,Tax Law,23019.82,34,63,6.1,18650.789343
2,Michael Lopez Jr.,Tax Law,19832.51,6,73,8.8,19428.601217
3,Mary Nichols,Tax Law,13597.30,7,155,4.2,20054.552636
4,Dr. Wayne Jackson,Tax Law,9342.90,26,6,4.2,20701.410942
5,Denise Thompson,Tax Law,20183.06,29,26,5.6,20762.558887
6,Christine Brown,Tax Law,29839.41,25,170,8.3,20847.659355
7,Matthew Gray,Tax Law,19545.63,31,81,4.1,20942.523588
8,Edward Zimmerman,Tax Law,25581.01,14,33,9.3,21101.123770
9,Kimberly Barry,Tax Law,18497.44,10,31,8.5,21420.538992


=== Corporate Law, Budget 50000 ===


,Name,Domain,Price,Years of active experience,No of favoured settlements,Client satisfaction (out of 10),Price Objective
0,Allison Cooper,Corporate Law,16336.11,13,16,7.3,23346.193934
1,Christopher Williams,Corporate Law,15413.83,30,26,4.7,23897.747252
2,Taylor Anderson,Corporate Law,4255.01,2,5,4.9,24108.001361
3,Richard Ruiz,Corporate Law,10569.22,3,64,5.5,24227.980857
4,Jodi Gallagher,Corporate Law,7882.93,14,31,4.2,25766.488827
5,Danielle Leon,Corporate Law,21402.89,22,20,6.6,25839.788402
6,Thomas Smith,Corporate Law,35082.81,11,217,8.8,26192.727447
7,Edward Lowe,Corporate Law,14919.76,9,51,4.7,26765.426760
8,Sara Harris,Corporate Law,31756.04,29,216,5.0,27191.671656
9,Timothy Henry,Corporate Law,34762.19,25,167,8.1,27313.527559
